In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time
import tensorflow as tf
from tensorflow import keras

F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
#分割数据集为：0.7训练集，0.1验证集，0.2测试集

from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(
        housing.data,
        housing.target,
        random_state = 7,
        test_size=0.2)

x_train,x_valid,y_train,y_valid = train_test_split(
        x_train_all,
        y_train_all,
        random_state=11,
        test_size=0.125)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

(14448, 8) (14448,)
(2064, 8) (2064,)
(4128, 8) (4128,)


In [4]:
#归一化操作  x = (s-u)/std   减去均值，除以方差

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


x_train_scaler = scaler.fit_transform(x_train)
x_valid_scaler = scaler.transform(x_valid)
x_test_scaler = scaler.transform(x_test)

In [6]:
'''
#函数式API

input = keras.layers.Input(shape = x_train.shape[1:])
hidden1 = keras.layers.Dense(100,activation='relu')(input)
hidden2 = keras.layers.Dense(100,activation='relu')(hidden1)

concat = keras.layers.concatenate([input,hidden2])
output = keras.layers.Dense(1)(concat)

model = keras.models.Model(inputs = [input], outputs = [output])
'''

#使用子类API
class WideAndDeep(keras.models.Model):
    def __init__(self):
        super(WideAndDeep,self).__init__()
        #定义模型
        self.hidden1_layer = keras.layers.Dense(30,activation='relu')
        self.hidden2_layer = keras.layers.Dense(30,activation='relu')
        self.output_layer = keras.layers.Dense(1)
        
    def call(self,input):
        #模型的正向计算
        hidden1 = self.hidden1_layer(input)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input,hidden2])
        output = self.output_layer(concat)
        return output

    
    
    
model = WideAndDeep()
model.build(input_shape=(None,8))




    
model.summary()
model.compile(loss="mean_squared_error",
             optimizer="sgd",
             metrics=["accuracy"])

Model: "wide_and_deep_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              multiple                  270       
_________________________________________________________________
dense_4 (Dense)              multiple                  930       
_________________________________________________________________
dense_5 (Dense)              multiple                  39        
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x_train_scaler, y_train, epochs = 100,
                   validation_data=(x_valid_scaler,y_valid),
                   
                   callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]
                   )

Train on 14448 samples, validate on 2064 samples
Epoch 1/100
14448/14448 [==============================] - 2s 155us/sample - loss: 0.7989 - accuracy: 0.0028 - val_loss: 0.4791 - val_accuracy: 0.0039
Epoch 2/100
14448/14448 [==============================] - 1s 76us/sample - loss: 0.7906 - accuracy: 0.0028 - val_loss: 0.4527 - val_accuracy: 0.0039
Epoch 3/100
14448/14448 [==============================] - 1s 76us/sample - loss: 1.0914 - accuracy: 0.0028 - val_loss: 0.5505 - val_accuracy: 0.0039
Epoch 4/100
14448/14448 [==============================] - 1s 78us/sample - loss: 1.7266 - accuracy: 0.0028 - val_loss: 1.2452 - val_accuracy: 0.0024
Epoch 5/100
14448/14448 [==============================] - 2s 108us/sample - loss: nan - accuracy: 0.0023 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/100
 1408/14448 [=>............................] - ETA: 1s - loss: nan - accuracy: 0.0000e+00

F:\Anaconda3\lib\site-packages\tensorflow\python\keras\callbacks.py:1224: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


14448/14448 [==============================] - 1s 100us/sample - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/100
14448/14448 [==============================] - 1s 76us/sample - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [8]:
model.evaluate(x_test_scaler,y_test)

4128/4128 [==============================] - 0s 40us/sample - loss: nan - accuracy: 0.0000e+00


[nan, 0.0]